In [425]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import warnings

df = pd.read_excel('atoll_2023.xlsx')
dfb = pd.read_excel('u2020.xlsx')
pd.set_option('display.max_columns',85)

In [426]:
df.head()

,COUNTY_ID,REGION,SITE_NAME,TX_ID,ANTENNA_NAME,SHAREDMAST,SHAREDPATTERN,AZIMUTH,HEIGHT,TILT,ELECTRICAL_TILT,ELECTRICAL_AZIMUTH,CONSTRUCTOR,FBAND,STATUS,SERIALNO,NEID,SUBNAME,SECTOR
0,SGS_WE,SGS,SGS_1001,SGS_E3_1001_1,065_16_XTB_1800_4_KATH_742241,2454642.0,336456.0,0.0,42.0,1.0,4.0,0.0,Kathrein_742241_1800,LTE 1800,Operational,NaN,NaN,NaN,NaN
1,SGS_WE,SGS,SGS_1001,SGS_P_1001_1,065_17_XTB_0900_4_KATH_742241,2454642.0,NaN,0.0,42.0,1.0,4.0,0.0,Kathrein_742241_900,GSM 1800,Operational,NaN,NaN,NaN,NaN
2,SGS_WE,SGS,SGS_1001,SGS_X_1001_1,065_17_XTB_0900_4_KATH_742241,2454642.0,NaN,0.0,42.0,1.0,4.0,0.0,Kathrein_742241_900,UMTS 900,Operational,NaN,NaN,NaN,NaN
3,SGS_WE,SGS,SGS_1001,SGS_E3_1001_2,065_16_XTB_1800_5_KATH_742241,2454643.0,336457.0,230.0,42.0,0.0,5.0,0.0,Kathrein_742241_1800,LTE 1800,Operational,NaN,NaN,NaN,NaN
4,SGS_WE,SGS,SGS_1001,SGS_P_1001_2,065_17_XTB_0900_5_KATH_742241,2454643.0,NaN,230.0,42.0,0.0,5.0,0.0,Kathrein_742241_900,GSM 1800,Operational,NaN,NaN,NaN,NaN


In [427]:
def antenna(df):
 
    antenna= df[['TX_ID','SITE_NAME','ANTENNA_NAME']].copy()
    antenna=antenna.drop_duplicates(subset=['TX_ID'])
    antenna['BW'] = df['ANTENNA_NAME'].astype(str).str[0:3]
    antenna['GAIN'] = df['ANTENNA_NAME'].astype(str).str[4:6]
    antenna['BAND'] = df['ANTENNA_NAME'].astype(str).str[7:10]
    

    
    antenna['LAYER'] = df['ANTENNA_NAME'].astype(str).str[11:15] 
    antenna.loc[antenna['LAYER'] == '800_', 'LAYER'] = '1800'
    antenna.loc[antenna['LAYER'] == '900_', 'LAYER'] = '900'
    antenna.loc[antenna['LAYER'] == '100_', 'LAYER'] = '2100'
    antenna = antenna.replace(['5dBi','i 0T','900M',''],'0')
    

    
    antenna['CODE'] = df['ANTENNA_NAME'].astype(str).str[17:42]
    antenna['CODE']=antenna['CODE'].str.lstrip('0123456._')
    return antenna

dfc=antenna(df)

In [428]:
dfc

,TX_ID,SITE_NAME,ANTENNA_NAME,BW,GAIN,BAND,LAYER,CODE
0,SGS_E3_1001_1,SGS_1001,065_16_XTB_1800_4_KATH_742241,065,16,XTB,1800,KATH_742241
1,SGS_P_1001_1,SGS_1001,065_17_XTB_0900_4_KATH_742241,065,17,XTB,0900,KATH_742241
2,SGS_X_1001_1,SGS_1001,065_17_XTB_0900_4_KATH_742241,065,17,XTB,0900,KATH_742241
3,SGS_E3_1001_2,SGS_1001,065_16_XTB_1800_5_KATH_742241,065,16,XTB,1800,KATH_742241
4,SGS_P_1001_2,SGS_1001,065_17_XTB_0900_5_KATH_742241,065,17,XTB,0900,KATH_742241
...,...,...,...,...,...,...,...,...
38977,SGS_X_9801_3_P,SGS_9801,063_16_XPB_0900_6_HUAW_AAU5951,063,16,XPB,0900,HUAW_AAU5951
38978,SGS_E3_TKSite14784_2,SGS_TKSite14784,068_16_XTB_1800_2_KATH_80010891,068,16,XTB,1800,KATH_80010891
38979,SGS_P_TKSite14784_2,SGS_TKSite14784,068_16_XTB_900_2_KATH_80010891,068,16,XTB,900,KATH_80010891
38980,SGS_E3_TKSite14784_3,SGS_TKSite14784,068_16_XTB_1800_2_KATH_80010891,068,16,XTB,1800,KATH_80010891


In [429]:
dfc[dfc['TX_ID']=='SGS_E8_14371_2']

,TX_ID,SITE_NAME,ANTENNA_NAME,BW,GAIN,BAND,LAYER,CODE
7166,SGS_E8_14371_2,SGS_14371,072_16_XDB_0900_6_CELW_APXV14,072,16,XDB,0900,CELW_APXV14


In [430]:
def summary(dfc):
    print(f"Dataset has {dfc.shape[1]} features and {dfc.shape[0]} examples.")
    summary = pd.DataFrame(index=dfc.columns)
    summary["Unique"] = dfc.nunique().values
    summary["Missing"] = dfc.isnull().sum().values
    summary["Duplicated"] = dfc.duplicated().sum()
    summary["Types"] = dfc.dtypes
    return summary

summary(dfc)

Dataset has 8 features and 38805 examples.


,Unique,Missing,Duplicated,Types
TX_ID,38805,0,0,object
SITE_NAME,1798,0,0,object
ANTENNA_NAME,874,0,0,object
BW,24,0,0,object
GAIN,20,0,0,object
BAND,15,0,0,object
LAYER,12,0,0,object
CODE,104,0,0,object


In [431]:
dfb

,REGION,COUNTRY,ENODEB,ENodeBId,LOCALCELL_ID,Urgency,Elec Tilt Warning,SiteID,TX_ID,Tilt,Planning Elec Tilt,CTRLCN,CTRLSN,CTRLSRN,RETTYPE,SCENARIO,SERIALNO,DEVICENAME,SUBNAME,SUBUNITNO
0,SGS,ZA,LTE_Brakpan_GSMT01_SGS,211041,1,Warning,Elec Tilt: Plan (4) vs RET (2),SGS_11041,SGS_E3_11041_1,2.0,4.0,0,0,80,SINGLE_RET,DAISY_CHAIN,E4L2444674-Y4,Sector 1 - WBS2600/WBS1800,NaN,1
1,SGS,ZA,LTE_City_Deep_West_GSMT01_SGS,235181,1,Critical,Elec Tilt: Plan (7) vs RET (2),SGS_35181,SGS_E3_35181_1,2.0,7.0,0,0,80,SINGLE_RET,DAISY_CHAIN,E4K4335814-Y4,Sector 1 - WBS2600/WBS1800,NaN,1
2,SGS,ZA,LTE_Empire_GSMT01_SGS,213651,1,Warning,Elec Tilt: Plan (5) vs RET (2.5),SGS_13651,SGS_E3_13651_1,2.5,5.0,0,0,80,SINGLE_RET,DAISY_CHAIN,CSJ0615898-Y4,Sector 1 - WBS2600/WBS1800,NaN,1
3,SGS,ZA,LTE_Khandubuhle_GSMT01_SGS,212361,1,Warning,Elec Tilt: Plan (7) vs RET (5),SGS_12361,SGS_E3_12361_1,5.0,7.0,0,0,80,SINGLE_RET,DAISY_CHAIN,E4K5081283-Y3,Sector 1 - VDC1800/VDC2100 - 2,NaN,1
4,SGS,ZA,LTE_Kitchener_Ave_GSMT01_SGS,234101,1,Critical,Elec Tilt: Plan (5) vs RET (14),SGS_34101,SGS_E3_34101_1,14.0,5.0,0,0,80,SINGLE_RET,DAISY_CHAIN,E4K4867214-Y4,Sector 1 - WBS2600/WBS1800,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13610,SGS,ZA,LTE_Vitafoam_DAS_GSMT01_SGS,259501,42,Critical,Elec Tilt: Plan (12) vs RET (5),SGS_59501,SGS_E8_59501_4,5.0,12.0,0,0,83,SINGLE_RET,DAISY_CHAIN,E4L2436040-R1,VDC_900Sec_4,VDC_900Sec_4,1
13611,SGS,ZA,LTE_Vitafoam_DAS_GSMT01_SGS,259501,50,Warning,Elec Tilt: Plan (14) vs RET (12),SGS_59501,SGS_E3_59501_5,12.0,14.0,0,0,84,SINGLE_RET,DAISY_CHAIN,E4L2436215-Y3,VDC_1800Sec_5,VDC_1800Sec_5,1
13612,SGS,ZA,LTE_Vitafoam_DAS_GSMT01_SGS,259501,50,Critical,Elec Tilt: Plan (14) vs RET (5),SGS_59501,SGS_E3_59501_5,5.0,14.0,0,0,84,SINGLE_RET,DAISY_CHAIN,E4L2436215-Y4,WBS_1800Sec_5,WBS_1800Sec_5,1
13613,SGS,ZA,LTE_Vitafoam_DAS_GSMT01_SGS,259501,51,Critical,Elec Tilt: Plan (14) vs RET (5),SGS_59501,SGS_E1_59501_5,5.0,14.0,0,0,84,SINGLE_RET,DAISY_CHAIN,E4L2436215-Y1,VDC_2100Sec_5,VDC_2100Sec_5,1


In [432]:
dfb[dfb['TX_ID'].duplicated()].sort_values(by=['TX_ID'],ascending=True).head(10)

,REGION,COUNTRY,ENODEB,ENodeBId,LOCALCELL_ID,Urgency,Elec Tilt Warning,SiteID,TX_ID,Tilt,Planning Elec Tilt,CTRLCN,CTRLSN,CTRLSRN,RETTYPE,SCENARIO,SERIALNO,DEVICENAME,SUBNAME,SUBUNITNO
12528,SGS,ZA,LTE_JG_Strydom_GSMT01_SGS,210011,31,Critical,Elec Tilt: Plan (6) vs RET (13),SGS_10011,SGS_E1_10011_3,13.0,6.0,0,0,82,SINGLE_RET,DAISY_CHAIN,E4L2445539-Y3,Sector 3 - VDC1800/VDC2100 - 2,NaN,1
12500,SGS,ZA,LTE_JHB_Central_GSMT01_SGS,210051,31,Critical,Elec Tilt: Plan (4) vs RET (14),SGS_10051,SGS_E1_10051_3,14.0,4.0,0,0,82,SINGLE_RET,DAISY_CHAIN,E4K2237284-Y3,Sector 3 - VDC1800/VDC2100 - 2,NaN,1
5226,SGS,ZA,LTE_Jeppe_GSMT01_SGS,210111,11,Urgent,Elec Tilt: Plan (6) vs RET (10),SGS_10111,SGS_E1_10111_1,10.0,6.0,0,0,80,SINGLE_RET,DAISY_CHAIN,M3249JB012K01LTy2,Sector 1 - VDC1800/VDC2100 - 1,NaN,1
8890,SGS,ZA,LTE_Jeppe_GSMT01_SGS,210111,21,Urgent,Elec Tilt: Plan (6) vs RET (10),SGS_10111,SGS_E1_10111_2,10.0,6.0,0,0,81,SINGLE_RET,DAISY_CHAIN,M3249JB01KC83RTy4,Sector 2 - VDC1800/VDC2100 - 2,NaN,1
12508,SGS,ZA,LTE_City_Deep_GSMT01_SGS,210221,31,Warning,Elec Tilt: Plan (11) vs RET (13),SGS_10221,SGS_E1_10221_3,13.0,11.0,0,0,72,SINGLE_RET,DAISY_CHAIN,E4K4442420-Y1,Sector 3 - VDC1800/VDC2100 - 1,NaN,1
12634,SGS,ZA,LTE_City_Deep_GSMT01_SGS,210221,31,Critical,Elec Tilt: Plan (11) vs RET (4),SGS_10221,SGS_E1_10221_3,4.0,11.0,0,0,82,SINGLE_RET,DAISY_CHAIN,0000CN10144189128,Sector 3 - UMTS 2100,NaN,1
4680,SGS,ZA,LTE_Alberton_GSMT01_SGS,210301,11,Urgent,Elec Tilt: Plan (6) vs RET (10),SGS_10301,SGS_E1_10301_1,10.0,6.0,0,0,80,SINGLE_RET,DAISY_CHAIN,M3249K7048G09LTy2,Sector 1 - VDC1800/VDC2100 - 1,NaN,1
9222,SGS,ZA,LTE_Alberton_GSMT01_SGS,210301,21,Critical,Elec Tilt: Plan (6) vs RET (11),SGS_10301,SGS_E1_10301_2,11.0,6.0,0,0,81,SINGLE_RET,DAISY_CHAIN,M3249K7069K63LTy2,Sector 2 - VDC1800/VDC2100 - 1,NaN,1
12736,SGS,ZA,LTE_Alberton_GSMT01_SGS,210301,31,Warning,Elec Tilt: Plan (6) vs RET (8),SGS_10301,SGS_E1_10301_3,8.0,6.0,0,0,82,SINGLE_RET,DAISY_CHAIN,M3249K9020094RTy4,Sector 3 - VDC1800/VDC2100 - 2,NaN,1
8885,SGS,ZA,LTE_Glenvista_GSMT01_SGS,210311,21,Warning,Elec Tilt: Plan (6) vs RET (5),SGS_10311,SGS_E1_10311_2,5.0,6.0,0,0,81,SINGLE_RET,DAISY_CHAIN,M3249JA00L003RTy4,Sector 2 - VDC1800/VDC2100 - 2,NaN,1


In [433]:
dfb[dfb['TX_ID']=='SGS_E3_13531_4']

,REGION,COUNTRY,ENODEB,ENodeBId,LOCALCELL_ID,Urgency,Elec Tilt Warning,SiteID,TX_ID,Tilt,Planning Elec Tilt,CTRLCN,CTRLSN,CTRLSRN,RETTYPE,SCENARIO,SERIALNO,DEVICENAME,SUBNAME,SUBUNITNO
13601,SGS,ZA,LTE_Bez_Valley_GSMT01_SGS,213531,4,Critical,Elec Tilt: Plan (9) vs RET (2),SGS_13531,SGS_E3_13531_4,2.0,9.0,0,0,83,SINGLE_RET,DAISY_CHAIN,M2334J100CY61RTy3,Sector 4 - WBS1800,NaN,1
13603,SGS,ZA,LTE_Bez_Valley_GSMT01_SGS,213531,4,Critical,Elec Tilt: Plan (9) vs RET (2),SGS_13531,SGS_E3_13531_4,2.0,9.0,0,0,83,SINGLE_RET,DAISY_CHAIN,M2334J100D059LTy2,Sector 4 - WBS2600/WBS1800,NaN,1


In [434]:
dfb[dfb['TX_ID']=='SGS_E8_14371_2']

,REGION,COUNTRY,ENODEB,ENodeBId,LOCALCELL_ID,Urgency,Elec Tilt Warning,SiteID,TX_ID,Tilt,Planning Elec Tilt,CTRLCN,CTRLSN,CTRLSRN,RETTYPE,SCENARIO,SERIALNO,DEVICENAME,SUBNAME,SUBUNITNO
9623,SGS,ZA,LTE_Residensia_GSMT01_SGS,214371,22,Urgent,Elec Tilt: Plan (6) vs RET (10),SGS_14371,SGS_E8_14371_2,10.0,6.0,0,0,81,SINGLE_RET,DAISY_CHAIN,M3249M2002X448Lr1,Sector 2 (900 r1),NaN,1
9840,SGS,ZA,LTE_Residensia_GSMT01_SGS,214371,22,Urgent,Elec Tilt: Plan (6) vs RET (10),SGS_14371,SGS_E8_14371_2,10.0,6.0,0,0,81,SINGLE_RET,DAISY_CHAIN,M3249M2002Y147Rr2,Sector 2 - VDC900,NaN,1


In [456]:
dfb.dtypes

REGION                 object
COUNTRY                object
ENODEB                 object
ENodeBId                int64
LOCALCELL_ID            int64
Urgency                object
Elec Tilt Warning      object
SiteID                 object
TX_ID                  object
Tilt                  float64
Planning Elec Tilt    float64
CTRLCN                  int64
CTRLSN                  int64
CTRLSRN                 int64
RETTYPE                object
SCENARIO               object
SERIALNO               object
DEVICENAME             object
SUBNAME                object
SUBUNITNO               int64
dtype: object

In [449]:
dfb[dfb['Tilt']==' ']

,REGION,COUNTRY,ENODEB,ENodeBId,LOCALCELL_ID,Urgency,Elec Tilt Warning,SiteID,TX_ID,Tilt,Planning Elec Tilt,CTRLCN,CTRLSN,CTRLSRN,RETTYPE,SCENARIO,SERIALNO,DEVICENAME,SUBNAME,SUBUNITNO


In [413]:
def result():
    result = pd.merge(dfc,dfb[['Tilt','TX_ID','REGION']],on='TX_ID', how='right')
    result=result.drop_duplicates(subset=['TX_ID'])
    result =result.fillna('0')
    result.rename(columns={'Tilt': 'TILT'}, inplace=True)
    #result=df['TILT'].replace('', np.nan, inplace=True)##
    #result=df.dropna(subset=['TILT'], inplace=True)##
    result['TILT']=result['TILT'].astype(str)
    result['TILT']=result['TILT'].replace(to_replace=r'.0', value=' ', regex=True)
    result['SHAREDPATTERN']=df['SHAREDPATTERN'].replace(np.nan,'0').astype(str)
    result['NEW_ANTENNA']= result[['BW','GAIN','BAND','LAYER','TILT','CODE']].agg("_".join, axis=1)
  
    
    return result
dr=result()

    

In [404]:
dr.head()

,TX_ID,SITE_NAME,ANTENNA_NAME,BW,GAIN,BAND,LAYER,CODE,TILT,REGION,SHAREDPATTERN,NEW_ANTENNA
0,SGS_E3_11041_1,SGS_11041,065_17_XTB_1800_4_KATH_80010892v01,065,17,XTB,1800,KATH_80010892v01,2,SGS,336456.0,065_17_XTB_1800_2_KATH_80010892v01
1,SGS_E3_35181_1,SGS_35181,065_17_XTB_1800_7_KATH_80010892v01,065,17,XTB,1800,KATH_80010892v01,2,SGS,0,065_17_XTB_1800_2_KATH_80010892v01
2,SGS_E3_13651_1,SGS_13651,065_17_XTB_1800_5_KATH_80010892v01,065,17,XTB,1800,KATH_80010892v01,2.5,SGS,0,065_17_XTB_1800_2.5_KATH_80010892v01
3,SGS_E3_12361_1,SGS_12361,065_17_XTB_1800_7_KATH_80010892v01,065,17,XTB,1800,KATH_80010892v01,5,SGS,336457.0,065_17_XTB_1800_5_KATH_80010892v01
4,SGS_E3_34101_1,SGS_34101,065_17_XTB_1800_5_KATH_80010892v01,065,17,XTB,1800,KATH_80010892v01,14,SGS,0,065_17_XTB_1800_14_KATH_80010892v01


In [405]:
#Tilt has no value

dr[dr['TX_ID']=='SGS_E8_14371_2']

,TX_ID,SITE_NAME,ANTENNA_NAME,BW,GAIN,BAND,LAYER,CODE,TILT,REGION,SHAREDPATTERN,NEW_ANTENNA
9623,SGS_E8_14371_2,SGS_14371,072_16_XDB_0900_6_CELW_APXV14,072,16,XDB,0900,CELW_APXV14,,SGS,0,072_16_XDB_0900__CELW_APXV14


In [406]:
def clean(dr):
    dr=dr.drop(['BW', 'GAIN', 'BAND', 'LAYER', 'CODE', 'TILT', 'REGION',],axis=1)
    return dr
dc=clean(dr)

In [407]:
dc.head(20)

,TX_ID,SITE_NAME,ANTENNA_NAME,SHAREDPATTERN,NEW_ANTENNA
0,SGS_E3_11041_1,SGS_11041,065_17_XTB_1800_4_KATH_80010892v01,336456.0,065_17_XTB_1800_2_KATH_80010892v01
1,SGS_E3_35181_1,SGS_35181,065_17_XTB_1800_7_KATH_80010892v01,0,065_17_XTB_1800_2_KATH_80010892v01
2,SGS_E3_13651_1,SGS_13651,065_17_XTB_1800_5_KATH_80010892v01,0,065_17_XTB_1800_2.5_KATH_80010892v01
3,SGS_E3_12361_1,SGS_12361,065_17_XTB_1800_7_KATH_80010892v01,336457.0,065_17_XTB_1800_5_KATH_80010892v01
4,SGS_E3_34101_1,SGS_34101,065_17_XTB_1800_5_KATH_80010892v01,0,065_17_XTB_1800_14_KATH_80010892v01
5,SGS_E3_26161_1,SGS_26161,063_16_XPB_1800_5_HUAW_AAU5951,0,063_16_XPB_1800_9_HUAW_AAU5951
6,SGS_E3_22361_1,SGS_22361,063_16_XPB_1800_7_HUAW_AAU5951,0,063_16_XPB_1800_2_HUAW_AAU5951
8,SGS_E3_7571_1,SGS_7571,065_17_XTB_1800_6_KATH_80010892v01,0,065_17_XTB_1800_2_KATH_80010892v01
9,SGS_E3_19601_1,SGS_19601,065_17_XTB_1800_6_KATH_80010892v01,0,065_17_XTB_1800_2_KATH_80010892v01
10,SGS_E3_18151_1,SGS_18151,072_16_XDB_1800_6_CELW_APXV14,0,072_16_XDB_1800_8_CELW_APXV14


In [457]:
def summary(dc):
    print(f"Dataset has {dc.shape[1]} features and {dc.shape[0]} examples.")
    summary = pd.DataFrame(index=dc.columns)
    summary["Unique"] = dc.nunique().values
    summary["Missing"] = dc.isnull().sum().values
    summary["Duplicated"] = dc.duplicated().sum()
    summary["Types"] = dc.dtypes
    return summary

summary(dc)

Dataset has 5 features and 8061 examples.


,Unique,Missing,Duplicated,Types
TX_ID,8061,0,0,object
SITE_NAME,1182,0,0,object
ANTENNA_NAME,468,0,0,object
SHAREDPATTERN,1073,0,0,object
NEW_ANTENNA,684,0,0,object


In [409]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8061 entries, 0 to 13614
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TX_ID          8061 non-null   object
 1   SITE_NAME      8061 non-null   object
 2   ANTENNA_NAME   8061 non-null   object
 3   SHAREDPATTERN  8061 non-null   object
 4   NEW_ANTENNA    8061 non-null   object
dtypes: object(5)
memory usage: 377.9+ KB


In [410]:
dc.to_excel('Atoll_From_JupyterNotebook.xlsx', index=False)